## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

# Connect to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

Mounted at /content/drive
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=WNLy6e3ojCfwSFUqYIPN4rbV5syXdTtmTiLCVF8y17Y&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiNtVBGds741qE2EALcXfWrPflgfu5r8D3MupHLivMEdliUNUHNQvw

Successfully saved authorization token.


In [2]:
import os
from glob import glob
!pip install shapely
!pip install geopandas
!pip install xarray
!pip install rioxarray
#!pip install wxee
!pip install geojson
!pip install geemap

import geemap
import geojson
import shapely as shp
import geopandas as gpd
#import wxee
import numpy as np
import matplotlib.pyplot as plt


     |████████████████████████████████| 1.0 MB 8.2 MB/s 
     |████████████████████████████████| 15.4 MB 28.3 MB/s 
     |████████████████████████████████| 6.3 MB 45.4 MB/s 
     |████████████████████████████████| 46 kB 2.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 362 kB/s 
  Created wheel for rioxarray: filename=rioxarray-0.9.0-py3-none-any.whl size=54395 sha256=8d382af9ceea3d4eda70c0fa681873f914a046cf996abe2e48d23a407e77df2d
  Stored in directory: /root/.cache/pip/wheels/f2/0d/f2/29da7ed804b71eb1317c3683b373f59d7c0c4c501117f2b144
Successfully built rioxarray
     |████████████████████████████████| 43 kB 827 kB/s 
     |████████████████████████████████| 25.3 MB 1.8 MB/s 
  Created wheel for wxee: filename=wxee-0.3.2-py3-none-any.whl size=37659 sha256=12b001e51868d4457e2f9037b56f5bac4148b603c3f5f9891ce8706c9fff603b
  Stored in directory: /root/.cach

## User-defined Functions

In [3]:
# Function to Mask Clouds
def maskCloud(image):
  cloudMask = image.select(['pixel_qa']).bitwiseAnd(1<<5).eq(0)
  cloudShadowMask = image.select(['pixel_qa']).bitwiseAnd(1<<3).eq(0)
  final_mask = cloudMask.And(cloudShadowMask)
  return image.updateMask(final_mask)

In [4]:
# Function to add water Index bands
def addIndex(image):
  ndwi = image.normalizedDifference(['green', 'nir']).rename(['ndwi'])
  # https://www.tandfonline.com/doi/abs/10.1080/01431169608948714
  
  mndwi = image.normalizedDifference(['green', 'swir1']).rename(['mndwi'])
  # https://www.tandfonline.com/doi/abs/10.1080/01431160600589179
  
  awei = image.expression("blue + 2.5*green - 1.5*(nir+swir1) - 0.25*swir2", {
    'blue': image.select(['blue']),
    'green': image.select(['green']),
    'nir': image.select(['nir']),
    'swir1': image.select(['swir1']),
    'swir2': image.select(['swir2'])
  }).rename(['awei'])
  # https://www.sciencedirect.com/science/article/pii/S0034425713002873
  
  wiFi = image.expression("1.7204 + 171*green +3*red - 70*nir - 45*swir1 - 71*swir2",{
    'green': image.select(['green']),
    'red': image.select(['red']),
    'nir': image.select(['nir']),
    'swir1': image.select(['swir1']),
    'swir2': image.select(['swir2'])
  }).rename(['wiFi'])
  # https://www.sciencedirect.com/science/article/abs/pii/S0034425715302753
  
  return image.addBands([ndwi, mndwi, awei, wiFi])


In [5]:
# Function of ostu's method
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)

    indices = ee.List.sequence(1, size)

    def iFunc(i):
      aCounts = counts.slice(0, 0, i) 
      aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
      aMeans = means.slice(0, 0, i)
      aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(),[0]).get([0]).divide(aCount)
      bCount = total.subtract(aCount)
      bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
      return aCount.multiply(aMean.subtract(mean).pow(2)) \
            .add(bCount.multiply(bMean.subtract(mean).pow(2)))

    # Compute between sum of squares, where each mean partitions the data
    bss = indices.map(iFunc)

    return means.sort(bss).get([-1])

In [6]:
# Function of Otsu's Method
def thresholding(image):
  
  #waterIndex = index
  bands = ['ndwi', 'mndwi', 'awei', 'wiFi']

  #Compute the histogram of the NIR band. (the mean and variance are only FYI)
  hist_ndwi = image.select([bands[-1]]).reduceRegion(
      reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
      geometry = roi.getInfo(),
      scale = 30,
      maxPixels = 10e12,
      bestEffort = True)
  
  thrh_ndwi = otsu(hist_ndwi.get(bands[-1]+'_histogram'))

  watermask_ndwi = image.select([bands[-1]]).gt(thrh_ndwi).rename(['watermask_'+bands[-1]]).selfMask()
  
  #water = image.select([index]).lt(threshold)
  return image.addBands([watermask_ndwi]) \
              .clip(roi) \
              .copyProperties(image) \
              .set('system:time_start', ee.Date.fromYMD(image.get('year'), 12, 31))

In [7]:
def thresholding2(image):
  #indices = ['ndwi', 'mndwi', 'awei', 'wiFi']
  #indices = ['wiFi']
  indices = waterIndices

  histograms = [image.select([i]).reduceRegion(
      reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
      geometry = roi.getInfo(),
      scale = 30,
      maxPixels = 10e12,
      bestEffort = True) for i in indices]

  thrhs = [otsu(h.get(i+'_histogram')) for h,i in zip(histograms,indices)]

  watermasks = [image.select([i]).gt(t).rename(['watermask_'+i]).selfMask() for i,t in zip(indices,thrhs)]

  #.addBands([watermasks[0], watermasks[1], watermasks[2], watermasks[3]]) 
  return image.addBands(watermasks)\
              .clip(roi) \
              .copyProperties(image) \
              .set('system:time_start', ee.Date.fromYMD(image.get('year'), 12, 31))

## Prepare ROI (Asia) for processing

### Load one-country parcels

In [8]:
# Load all parcels
os.chdir('/content/drive/MyDrive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Coastline_parcels_polygon')
path_parcels = glob(os.path.join(os.getcwd(), '*.geojson'))[0]
parcels = gpd.read_file(path_parcels)

In [9]:
#parcels.to_file('parcels.geojson', driver='GeoJSON')
with open('parcels.geojson') as f:
  parcels_geojson = geojson.load(f)

In [10]:
#len(parcels_geojson['features'])

In [11]:
# List all countries
#countries = sorted(list(set(parcels['NAME_0'])))
#print(countries)

In [12]:
# Subset Parcels to one Country
#parcels_one_country = parcels[parcels['NAME_0']==countries[0]]
#parcels_one_country

## GEE: export data coverage /asia

In [ ]:
ROI = ee.FeatureCollection(parcels_geojson)

In [ ]:
# Load Landsat Archives
ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterDate('2013', '2020') \
        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
        .filterBounds(ROI) \
        .sort("system:time_start", True)
ls7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
      .filterDate('1999', '2020') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(ROI) \
      .sort("system:time_start", True)
ls5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
      .filterDate('1984', '2013') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(ROI) \
      .sort("system:time_start", True)
ls4 = ee.ImageCollection("LANDSAT/LT04/C01/T1_SR") \
      .filterDate('1984', '1994') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(ROI)\
      .sort("system:time_start", True)

# Merge Collections + Mask Cloud + Add water index bands
ls_all = ls8.merge(ls7).merge(ls5).merge(ls4)
ls_all = ls_all.sort('system:time_start', True).map(maskCloud).map(addIndex)

In [ ]:
# Count Observations at pixel level by year 
years = ee.List.sequence(1984, 2019)
counts = ee.ImageCollection.fromImages(
  years.map(lambda YEAR: 
            ls_all.select(['wiFi']).filter(ee.Filter.calendarRange(YEAR,YEAR,'year')).count().clip(ROI).set({'year':YEAR})))

In [ ]:
#Map = geemap.Map(center=(20, 110), zoom=3)
#Map.addLayer(ROI, {}, 'ROI')

#vis = {'palette':["#ebedef", "#f6ddcc", "#f8c471", "#f4d03f", "#58d68d", \
#                "#27ae60", "#1abc9c", "#3498db", "#2471a3", "#6c3483"], 'min':0, 'max':29}
#Map.addLayer(counts.filter(ee.Filter.eq('year',1987)), vis, '1987')
#Map.addLayer(counts.filter(ee.Filter.eq('year',2019)), vis, '2019')
#Map

In [ ]:
#img_count = counts.toBands()
#task_config = {
#    'description': 'data_coverage_asia_1km',
#    'fileNamePrefix': 'data_coverage_asia_1km',
#    'crs': 'EPSG:4326',
#    'scale': 1000,  
#    'region': ROI.geometry(),#.getInfo()['coordinates'],
#    'folder': 'myExportImage',
#    'skipEmptyTiles': True,
#    'fileFormat': 'GeoTIFF',
#    'maxPixels': 10e12
#    }
#task = ee.batch.Export.image.toDrive(img_count, **task_config)
#task.start() 
#task2 = ee.batch.Export.image.toDrive(img_count, **task_config)
#task2.start() 


In [ ]:
print(task.status())
print(task2.status())

{'state': 'COMPLETED', 'description': 'data_coverage_asia', 'creation_timestamp_ms': 1638907056438, 'update_timestamp_ms': 1638909341789, 'start_timestamp_ms': 1638907140885, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_'], 'attempt': 1, 'id': 'NMMRUPAMGSTIMES332YBSKW3', 'name': 'project

## GEE: export water masks & data coverage /parcel

In [13]:
# Define roi
roi = ee.Geometry(parcels_geojson['features'][100]['geometry'])
roi.getInfo()

{'coordinates': [[[117.343127, 21.713234],
   [115.604848, 24.620507],
   [114.259554, 24.219888],
   [115.30293, 21.028375],
   [117.343127, 21.713234]]],
 'type': 'Polygon'}

In [14]:
# Load Landsat Archives
ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterDate('2013', '2020') \
        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
        .filterBounds(roi) \
        .sort("system:time_start", True)
ls7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
      .filterDate('1999', '2020') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(roi) \
      .sort("system:time_start", True)
ls5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
      .filterDate('1984', '2013') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(roi) \
      .sort("system:time_start", True)
ls4 = ee.ImageCollection("LANDSAT/LT04/C01/T1_SR") \
      .filterDate('1984', '1994') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(roi)\
      .sort("system:time_start", True)

# Merge Collections + Mask Cloud + Add water index bands
ls_all = ls8.merge(ls7).merge(ls5).merge(ls4)
ls_all = ls_all.sort('system:time_start', True).map(maskCloud).map(addIndex)

In [41]:
# Temporally reduced at scale of one year
years = ee.List.sequence(1984, 2019)

ls_Treduced = ee.ImageCollection.fromImages(
    years.map(lambda YEAR: ls_all.filter(ee.Filter.calendarRange(YEAR, YEAR, 'year')) \
                          .median() \
                          .clip(roi) \
                          .set({'year': YEAR, 'system:time_start': ee.Date.fromYMD(YEAR, 12, 31)})))

In [42]:
# Data Availability: annual counts of observations
counts = ee.ImageCollection.fromImages(
  years.map(lambda YEAR: 
            ls_all.select(['wiFi'], ['count']).filter(ee.Filter.calendarRange(YEAR,YEAR,'year')).count().clip(roi).set({'year':YEAR, 'system:time_start': ee.Date.fromYMD(YEAR, 12, 31)})))
print(counts.size().getInfo())

36


In [43]:
# Filter out years where no data are available
# Add "bandlength" as new property
ls_filtered = ls_Treduced.map(lambda image: image.set('bandlength', image.bandNames().size()))
# Filter out the years where no data are available
ls_filtered = ls_filtered.filterMetadata('bandlength', 'not_equals', 0)

In [59]:
# Apply Otsu
#waterIndices = ['ndwi', 'mndwi', 'awei', 'wiFi']
waterIndices = ['wiFi']

ls_wmsk = ls_filtered.map(thresholding2)
ls_wmsk = ls_wmsk.select(['watermask_'+i for i in waterIndices]) 

print(ls_wmsk.size().getInfo())
print(ls_wmsk.first().bandNames().getInfo())

34
['watermask_wiFi']


In [60]:
# Join Two ImageCollections: water masks and counts
# https://gis.stackexchange.com/questions/327904/combine-two-image-collections-into-one-image-collection-earth-engine
# https://gis.stackexchange.com/questions/236551/combine-bands-from-two-image-collections

filter = ee.Filter.equals(leftField = 'year', rightField = 'year')
innerJoin = ee.ImageCollection(ee.Join.inner().apply(ls_wmsk, counts, filter))
ls_joined = innerJoin.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary')))\
                  .map(lambda image: image.toByte()) # Cast to uint-8bit

print(ls_joined.size().getInfo())
print(ls_joined.first().bandNames().getInfo())
print(ls_joined.first().bandTypes().getInfo())

34
['watermask_wiFi', 'count']
{'count': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'watermask_wiFi': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}


In [61]:
Map = geemap.Map(center=(20, 110), zoom=3)
#Map.addLayer(roi, {}, 'roi', False)
#Map.addLayer(ls_filtered.filter(ee.Filter.eq('year', 2019)), {'bands':['nir', 'red', 'green'], min:0, max:2550}, 'FCC')
#Map.addLayer(ls_wmsk.filter(ee.Filter.eq('year', 2019)).select(['watermask_mndwi']), {'palette':'blue'}, 'water mask mndwi')
#Map.addLayer(ls_wmsk.filter(ee.Filter.eq('year', 2019)).select(['watermask_awei']), {'palette':'orange'}, 'water mask awei')
Map.addLayer(ls_joined.select(['watermask_wiFi']).filter(ee.Filter.eq('year', 2019)), {'palette':'pink'}, 'water mask wiFi')

vis = {'palette':["#ebedef", "#f6ddcc", "#f8c471", "#f4d03f", "#58d68d", \
                "#27ae60", "#1abc9c", "#3498db", "#2471a3", "#6c3483"], 'min':0, 'max':29}
#Map.addLayer(counts.filter(ee.Filter.eq('year',1987)), vis, '1987')
#Map.addLayer(counts.filter(ee.Filter.eq('year',2019)), vis, '2019')
Map.addLayer(ls_joined.select(['count']).filter(ee.Filter.eq('year', 2019)), vis, '2019')
Map

Map(center=[20, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [62]:
img_joined = ls_joined.toBands()
task_config = {
    'description': 'p100_maskx1_countx1',
    'fileNamePrefix': 'p100_maskx1_countx1',
    'crs': 'EPSG:4326',
    'scale': 30,  
    'region': roi.getInfo()['coordinates'],
    'folder': 'myExportImage',
    'skipEmptyTiles': True,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 10e12
    }
#task3 = ee.batch.Export.image.toDrive(img_joined, **task_config)
#task3.start() 

In [80]:
task3.status()

{'attempt': 1,
 'creation_timestamp_ms': 1638992996925,
 'description': 'p100_maskx1_countx1',
 'id': 'SQHV2DNCXNFSNIUKQRC4NWGN',
 'name': 'projects/earthengine-legacy/operations/SQHV2DNCXNFSNIUKQRC4NWGN',
 'start_timestamp_ms': 1638993040323,
 'state': 'RUNNING',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1638995705360}

In [40]:
tasks = geetools.batch.Export.imagecollection.toDrive(
            collection = ls_joined,
            folder = 'myExportImage',
            region = roi,
            namePattern = 'p100_maskx1_countx1',
            scale = 30,
            dataType = 'uint8',
            #datePattern=date_pattern,
            #extra=extra,
            verbose=True,
            maxPixels=int(1e13)
        )

EEException: ignored

In [31]:
!pip install geetools
import geetools

     |████████████████████████████████| 74 kB 2.3 MB/s 
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92105 sha256=9893bc98ead1b78f1c0a990e4f39529bc1e6fbb0fec5f11bd1fb67880cf42884
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built geetools


## Export GEE Map to html / image